# Contrast coding

## What and why?

To fit any kind of statistical model you need some kind of *numerical
representation* of your data. Data often comes in a *table*, a named
collection of variables of different types of data. Some of that data is
"continuous", or basically numeric. But often our data is not numeric
(or continuous), but "categorical", having a finite number of distinct
levels.

For instance, let's look at the KB07 dataset:

In [44]:
using DataFrames, DataFramesMeta
using MixedModels, GLM
using LinearAlgebra

kb07 = MixedModels.dataset(:kb07)
first(kb07, 5)

,subj,item,spkr,prec,load,rt_trunc,rt_raw
,String,String,String,String,String,Int16,Int16
1,S030,I01,new,break,yes,2267,2267
2,S030,I02,old,maintain,no,3856,3856
3,S030,I03,old,break,no,1567,1567
4,S030,I04,new,maintain,no,1732,1732
5,S030,I05,new,break,no,2660,2660


Here `:spkr`, `:prec`, and `:load` are categortical variables, each of
which takes on two different values. If we fit a regression using this
dataset, we end up with predictors that refer to specific levels:

In [45]:
f = @formula(rt_trunc ~ 1 + spkr + prec + spkr&prec + (1 | subj))
mod = fit(MixedModel, f, kb07)

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr + prec + spkr & prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
──────────────────────────────────────────────────────────────────
                             Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────────────────────
(Intercept)                 2425.32      56.5224    42.91   <1e-99
spkr: old                    179.992     54.4214     3.31   0.0009
prec: maintain              -623.347     54.4214   -11.45   <1e-29
spkr: old & prec: maintain   -86.7763    76.9856    -1.13   0.2597
──────────────────────────────────────────────────────────────────

Let's look at a few rows of the fixed effects design matrix that's
generated for this model:

In [46]:
Int.(mod.X)[1:5, :]

5×4 Array{Int64,2}:
 1  0  0  0
 1  1  1  1
 1  1  0  0
 1  0  1  0
 1  0  0  0

A few things to note: all the values are 0 or 1, and there's one column
of all 1s at the start (that's the `(Intercept)` term). Columns 2 and 3
correspond to `spkr` and `prec`: there's a 0 where `spkr == "new"` and a
1 for `"old"`. Note that the coefficient name for this column is `spkr:
old`, which indicates that this predictor indicates the presence of
"old", relative to the (implicit) baseline of "new". Similarly for
`prec: maintain`.

The last column is the interaction term `spkr&prec`, and it's the
elementwise product of the columns for `spkr: new` and `pred: maintain`.

In [47]:
kb07[1:5, :]

,subj,item,spkr,prec,load,rt_trunc,rt_raw
,String,String,String,String,String,Int16,Int16
1,S030,I01,new,break,yes,2267,2267
2,S030,I02,old,maintain,no,3856,3856
3,S030,I03,old,break,no,1567,1567
4,S030,I04,new,maintain,no,1732,1732
5,S030,I05,new,break,no,2660,2660


## How to take control

You can set your own contrasts via the `contrasts=` keyword argument in
`fit`, with the variable you want to code as the key and contrasts as
the value:

In [48]:
using StatsModels

contrasts = Dict(
    :spkr => EffectsCoding(base = "old"),
    :prec => DummyCoding(levels = ["maintain", "break"])
)

mod2 = fit(MixedModel, f, kb07, contrasts=contrasts)

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr + prec + spkr & prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
───────────────────────────────────────────────────────────────
                          Estimate  Std.Error  z value  P(>|z|)
───────────────────────────────────────────────────────────────
(Intercept)              1848.58      49.5329    37.32   <1e-99
spkr: new                 -46.6081    27.2263    -1.71   0.0869
prec: break               666.735     38.4928    17.32   <1e-66
spkr: new & prec: break   -43.3882    38.4928    -1.13   0.2597
───────────────────────────────────────────────────────────────

This example illustrates two ways to control the ordering of levels used
to compute the contrasts:

1.  you can use `base=` to determine the baseline level
2.  you can use `levels=` to indicate all the levels that are used in
    the contrasts, the first of which is automatically set as the
    baseline.

## Built in contrast coding schemes

StatsModels.jl provides a few commonly used contrast coding schemes,
some less-commonly used schemes, and structs that allow you to manually
specify your own, custom schemes.

All are subtypes of the `AbstractContrasts` type:

In [49]:
using InteractiveUtils
subtypes(AbstractContrasts)

7-element Array{Any,1}:
 ContrastsCoding            
 DummyCoding                
 EffectsCoding              
 HelmertCoding              
 HypothesisCoding           
 SeqDiffCoding              
 StatsModels.FullDummyCoding

And all have fairly extensive documentation via the normal help system.
For instance:

In [50]:
# use ?SeqDiffCoding in the REPL
(@doc SeqDiffCoding).content

1-element Array{Any,1}:
 ```
SeqDiffCoding([base[, levels]])
```

Code each level in order to test "sequential difference" hypotheses, which compares each level to the level below it (starting with the second level). Specifically, the $n$th predictor tests the hypothesis that the difference between levels $n$ and $n+1$ is zero.

# Examples

```jldoctest seqdiff
julia> seqdiff = StatsModels.ContrastsMatrix(SeqDiffCoding(), ["a", "b", "c", "d"]).matrix
4×3 Array{Float64,2}:
 -0.75  -0.5  -0.25
  0.25  -0.5  -0.25
  0.25   0.5  -0.25
  0.25   0.5   0.75
```

The interpretation of sequential difference coding may be hard to see from the contrasts matrix itself.  The corresponding hypothesis matrix shows a clearer picture.  From the rows of the hypothesis matrix, we can see that these contrasts test the difference between the first and second levels, the second and third, and the third and fourth, respectively:

```jldoctest seqdiff
julia> round.(pinv(seqdiff), digits=2)
3×4 Array{Float64,2

### Standard contrasts

The most commonly used contrasts are `DummyCoding` and `EffectsCoding`
(which are similar to `contr.treatment` and `contr.sum` in R,
respectively).

### "Exotic" contrasts

We also provide `HelmertCoding` and `SeqDiffCoding` (corresponding to
base R's `contr.helmert` and MASS's `contr.sdiff`).

### Manual contrasts

There are two ways to manually specify contrasts. First, you can specify
them **directly** via `ContrastsCoding`. If you do, it's good practice
to specify the levels corresponding to the rows of the matrix, although
they can be omitted in which case they'll be inferred from the data.

For instance, here's a weird set of contrasts for `:spkr`:

In [51]:
cs = Matrix([-1/3 2/3]')
contr_manual = Dict(:spkr => StatsModels.ContrastsCoding(cs, levels=["old", "new"]))
mod3 = fit(MixedModel, f, kb07, contrasts=contr_manual)

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr + prec + spkr & prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
──────────────────────────────────────────────────────────────────
                             Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────────────────────
(Intercept)                 2545.31      50.3425    50.56   <1e-99
spkr: new                   -179.992     54.4214    -3.31   0.0009
prec: maintain              -681.198     40.582    -16.79   <1e-62
spkr: new & prec: maintain    86.7763    76.9856     1.13   0.2597
──────────────────────────────────────────────────────────────────

(Note that the estimates and even the signs of the fixed effect βs
change when we change the contrasts, but the overall log-likelihood
doesn't).

We can see that the values from the contrasts matrix we specified are
plugged directly in to the fixed effects matrix, and are also used in
computing the predictor for the interaction:

In [52]:
mod3.X[1:5, :]

5×4 Array{Float64,2}:
 1.0   0.666667  0.0   0.0     
 1.0  -0.333333  1.0  -0.333333
 1.0  -0.333333  0.0  -0.0     
 1.0   0.666667  1.0   0.666667
 1.0   0.666667  0.0   0.0     

## Contrasts from hypotheses

A better way to specify manual contrasts is via `HypothesisCoding`,
where each row of the matrix corresponds to the weights given to the
cell means of the levels corresponding to each column (see [Schad et
al. 2020](https://doi.org/10.1016/j.jml.2019.104038) for more
information). This is less interesting with only two levels, so let's
look at a scenario where we combine `:spkr` and `:prec` into a single,
4-level predictor, and want to test some strange hypotheses.

In [53]:
kb07ex = @transform(kb07, spkr_prec = :spkr .* "-" .* :prec);

f2 = @formula(rt_trunc ~ 1 + spkr_prec + (1 | subj))
mod4 = fit(MixedModel, f2, kb07ex)

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr_prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
──────────────────────────────────────────────────────────────
                         Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────────────────
(Intercept)              2425.32     56.5224    42.91   <1e-99
spkr_prec: new-maintain  -623.347    54.4214   -11.45   <1e-29
spkr_prec: old-break      179.992    54.4214     3.31   0.0009
spkr_prec: old-maintain  -530.131    54.4839    -9.73   <1e-21
──────────────────────────────────────────────────────────────

Let's say we want to test whether the effect of `:prec` depends on
whether `:spkr` is old vs. new. We need one contrast to test the
hypothesis that `"maintain" != "break"` for "new", and another for
"old". That leaves one over, to test the overall difference between
"new" and "old".

In [54]:
levels = ["new-break", "new-maintain", "old-break", "old-maintain"]

prec_old = (levels .== "old-break") .- (levels .== "old-maintain")
prec_new = (levels .== "new-break") .- (levels .== "new-maintain")
old_new = (abs.(prec_old) .- abs.(prec_new)) ./ 2

contr_hyp = HypothesisCoding(Matrix(hcat(old_new, prec_old, prec_new)'),
                             labels=["old", "(old) break", "(new) break"])
contr_hyp.hypotheses

3×4 Array{Float64,2}:
 -0.5  -0.5  0.5   0.5
  0.0   0.0  1.0  -1.0
  1.0  -1.0  0.0   0.0

These hypotheses correspond to the following *contrasts* (using the
[`rationalize`](https://docs.julialang.org/en/v1/base/math/#Base.rationalize)
function to make pretty fractions, like the `fractions` function in R):

In [55]:
rationalize.(contr_hyp.contrasts, tol=1e-10)

4×3 Array{Rational{Int64},2}:
 -1//2   0//1   1//2
 -1//2   0//1  -1//2
  1//2   1//2   0//1
  1//2  -1//2   0//1

Notice how the ±1 coding in the hypotheses (which translates into the difference
between the mean response in those cells) is transformed into ±½ coding in the
contrasts.

In [56]:
mod5 = fit(MixedModel, f2, kb07ex, contrasts = Dict(:spkr_prec => contr_hyp))

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr_prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
─────────────────────────────────────────────────────────────
                        Estimate  Std.Error  z value  P(>|z|)
─────────────────────────────────────────────────────────────
(Intercept)             2181.94     45.6362    47.81   <1e-99
spkr_prec: old           136.604    38.4928     3.55   0.0004
spkr_prec: (old) break   710.123    54.4527    13.04   <1e-38
spkr_prec: (new) break   623.347    54.4214    11.45   <1e-29
─────────────────────────────────────────────────────────────

Note that this is equivalent to the `/` "nesting" syntax using `EffectsCoding`,
after adjusting for the 2× factor from the +1/-1 coding:

In [57]:
mod6 = fit(MixedModel,
           @formula(rt_trunc ~ 1 + spkr/prec + (1|subj)), 
           kb07,
           contrasts = Dict(:spkr => EffectsCoding(base="new"),
                            :prec => EffectsCoding(base="maintain")))

Linear mixed model fit by maximum likelihood
 rt_trunc ~ 1 + spkr + spkr & prec + (1 | subj)
     logLik        -2 logLik          AIC             BIC       
 -1.45767208×10⁴  2.91534417×10⁴  2.91654417×10⁴  2.91983781×10⁴

Variance components:
            Column    Variance  Std.Dev. 
subj     (Intercept)   95885.39 309.65366
Residual              662657.47 814.03776
 Number of obs: 1789; levels of grouping factors: 56

  Fixed-effects parameters:
───────────────────────────────────────────────────────────────
                          Estimate  Std.Error  z value  P(>|z|)
───────────────────────────────────────────────────────────────
(Intercept)              2181.94      45.6362    47.81   <1e-99
spkr: old                  68.3021    19.2464     3.55   0.0004
spkr: new & prec: break   311.673     27.2107    11.45   <1e-29
spkr: old & prec: break   355.062     27.2263    13.04   <1e-38
───────────────────────────────────────────────────────────────

# The `@formula`

A formula in Julia is created with the `@formula` macro.  Between the macro and
fitting a model, the formula goes through a number of steps.

* The `@formula` macro itself does some transformations on the syntax, and
  creates `Term`s
* Then a `Schema` is extracted from the data, which says which `Term`s are
  `ContinuousTerm`s and which are `CategoricalTerm`s
* The `Schema` is then used to transform the original formula into a "concrete
  formula".
* The concrete formula (with all `Term`s replaced by continuous/categorical
  versions) generates model matrix columns when given some data.

The details are described in the
[documentation](https://juliastats.org/StatsModels.jl/stable/internals/#The-lifecycle-of-a-@formula-1),
and for the most part modeling packages handle these steps for you.  But in the
interest of allowing you to do your own weird things, here are a few examples.

## A formula is made of terms

In [58]:
f = @formula(y ~ 1 + a + b + a&b)

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  a(unknown) & b(unknown)

You can inpsect the internal structure with (it's like `str` in R):

In [59]:
dump(f)

FormulaTerm{Term,Tuple{ConstantTerm{Int64},Term,Term,InteractionTerm{Tuple{Term,Term}}}}
  lhs: Term
    sym: Symbol y
  rhs: Tuple{ConstantTerm{Int64},Term,Term,InteractionTerm{Tuple{Term,Term}}}
    1: ConstantTerm{Int64}
      n: Int64 1
    2: Term
      sym: Symbol a
    3: Term
      sym: Symbol b
    4: InteractionTerm{Tuple{Term,Term}}
      terms: Tuple{Term,Term}
        1: Term
          sym: Symbol a
        2: Term
          sym: Symbol b


We can build the same formula directly, using terms:

In [60]:
t_a = term(:a)
t_b = term(:b)
t_1 = term(1)
t_y = term(:y)

dump(t_a)

Term
  sym: Symbol a


In [61]:
f_dir = FormulaTerm(t_y, (t_1, t_a, t_b, InteractionTerm((t_a, t_b))))

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  a(unknown) & b(unknown)

Or, using operator overloading:

In [62]:
f_op = t_y ~ t_1 + t_a + t_b + t_a & t_b

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  a(unknown) & b(unknown)

These three are all equivalent:

In [63]:
f == f_dir == f_op

true

## The schema gives concrete terms

If we have some fake data:

In [64]:
df = DataFrame(y = rand(100), a = rand(100), b = repeat([:Q, :R, :S, :T], 25))
first(df, 5)

,y,a,b
,Float64,Float64,Symbol
1,0.956405,0.338148,Q
2,0.929902,0.376832,R
3,0.747754,0.848059,S
4,0.573026,0.0967081,T
5,0.146775,0.92813,Q


We can extract a `Schema`:

In [65]:
sch = schema(df)

StatsModels.Schema with 3 entries:
  a => a
  y => y
  b => b


This maps un-typed `Term`s to concrete verisons.  Now we know that `:a` is a
continuous variable in this dataset:

In [66]:
sch[term(:a)]

a(continuous)

Categorical terms hold the contrasts matrix and levels:

In [67]:
t_b_concrete = sch[term(:b)]

b(DummyCoding:4→3)

In [68]:
cmat = t_b_concrete.contrasts
cmat.matrix

4×3 Array{Float64,2}:
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [69]:
cmat.levels

4-element Array{Symbol,1}:
 :Q
 :R
 :S
 :T

## `apply_schema` combines terms and schema to get concrete versions

The canonical case is to apply the schema to the whole formula:

In [70]:
apply_schema(f, sch)

FormulaTerm
Response:
  y(continuous)
Predictors:
  1
  a(continuous)
  b(DummyCoding:4→3)
  a(continuous) & b(DummyCoding:4→3)

Note that the schema gets pushed through the interaction term, too.

We can also apply the schema to a single term:

In [71]:
apply_schema(term(:a) & term(:b), sch)

a(continuous) & b(DummyCoding:4→3)

Of course if the schema doesn't have enough information, we'll get an error:

In [72]:
apply_schema(term(:argle_bargle), sch)

KeyError: KeyError: key argle_bargle not found

## Concrete terms generate arrays with `modelcols`

Any term can generate model columns with `modelcols`:

In [73]:
t_ab_concrete = apply_schema(term(:a) & term(:b), sch)
modelcols(t_ab_concrete, first(df, 6))

6×3 Array{Float64,2}:
 0.0       0.0       0.0      
 0.376832  0.0       0.0      
 0.0       0.848059  0.0      
 0.0       0.0       0.0967081
 0.0       0.0       0.0      
 0.211993  0.0       0.0      

Compare with:

In [74]:
f_concrete = apply_schema(f, sch)
@show t_ab_concrete_formula = f_concrete.rhs.terms[end]
modelcols(t_ab_concrete_formula, first(df, 6))

t_ab_concrete_formula = f_concrete.rhs.terms[end] = a & b


6×3 Array{Float64,2}:
 0.0       0.0       0.0      
 0.376832  0.0       0.0      
 0.0       0.848059  0.0      
 0.0       0.0       0.0967081
 0.0       0.0       0.0      
 0.211993  0.0       0.0      

Of course you can generate columns for the whole formula (it returns a tuple of
left-hand side, right-hand side columns):

In [75]:
y, X = modelcols(f_concrete, first(df, 6))
X

6×8 Array{Float64,2}:
 1.0  0.338148   0.0  0.0  0.0  0.0       0.0       0.0      
 1.0  0.376832   1.0  0.0  0.0  0.376832  0.0       0.0      
 1.0  0.848059   0.0  1.0  0.0  0.0       0.848059  0.0      
 1.0  0.0967081  0.0  0.0  1.0  0.0       0.0       0.0967081
 1.0  0.92813    0.0  0.0  0.0  0.0       0.0       0.0      
 1.0  0.211993   1.0  0.0  0.0  0.211993  0.0       0.0      

### Predicting based on new data

Any table with the right columns can be passed to `modelcols` and the right
columns are generated, even if some levels are missing:

In [76]:
df2 = DataFrame(a = rand(5), b = [:R, :R, :Q, :S, :R])
modelcols(f_concrete.rhs, df2)

5×8 Array{Float64,2}:
 1.0  0.800447    1.0  0.0  0.0  0.800447  0.0         0.0
 1.0  0.573827    1.0  0.0  0.0  0.573827  0.0         0.0
 1.0  0.801419    0.0  0.0  0.0  0.0       0.0         0.0
 1.0  0.00118095  0.0  1.0  0.0  0.0       0.00118095  0.0
 1.0  0.27748     1.0  0.0  0.0  0.27748   0.0         0.0

### Use a named tuple for a single row

A single row of the model matrix can be generated from a `NamedTuple` of data:

In [77]:
data_row = (a = 1.5, b = :T)
modelcols(f_concrete.rhs, data_row)

8-element Array{Float64,1}:
 1.0
 1.5
 0.0
 0.0
 1.0
 0.0
 0.0
 1.5

## Get coefficient names for any term with `coefnames`

In [78]:
coefnames(f_concrete)

("y", ["(Intercept)", "a", "b: R", "b: S", "b: T", "a & b: R", "a & b: S", "a & b: T"])

In [79]:
coefnames(f_concrete.rhs.terms[end])

3-element Array{String,1}:
 "a & b: R"
 "a & b: S"
 "a & b: T"

In [80]:
coefnames(sch[term(:b)])

3-element Array{String,1}:
 "b: R"
 "b: S"
 "b: T"

## Formula syntax

The formula syntax is very similar to R, with the exception that an interaction
is specified with `&`, and that some R syntax is not supported by default (`^`,
`/` outside of MixedModels.jl).

### Non-special calls 

Any function calls that are not special syntax (`+`, `&`, `*`, and `~`) are
treated as normal julia code, so you can write things like

In [81]:
f2 = @formula(log(y) ~ 1 + (a + a^2) * b)

FormulaTerm
Response:
  (y)->log(y)
Predictors:
  1
  a(unknown)
  (a)->a ^ 2
  b(unknown)
  a(unknown) & b(unknown)
  (a)->a ^ 2 & b(unknown)

In [82]:
f2_concrete = apply_schema(f2, sch)

FormulaTerm
Response:
  (y)->log(y)
Predictors:
  1
  a(continuous)
  (a)->a ^ 2
  b(DummyCoding:4→3)
  a(continuous) & b(DummyCoding:4→3)
  (a)->a ^ 2 & b(DummyCoding:4→3)

In [83]:
y2, X2 = modelcols(f2_concrete, first(df, 5))

([-0.04457330177727352, -0.07267575264362239, -0.2906812305720375, -0.5568250412666073, -1.918851244049599], [1.0 0.3381483054241785 … 0.0 0.0; 1.0 0.3768317547009774 … 0.0 0.0; … ; 1.0 0.09670814004285089 … 0.0 0.00935246435054766; 1.0 0.9281295827279596 … 0.0 0.0])

In [84]:
y2 == log.(df[1:5, :y])

true

In [85]:
X2

5×12 Array{Float64,2}:
 1.0  0.338148   0.114344    0.0  0.0  0.0  …  0.0       0.0       0.0       
 1.0  0.376832   0.142002    1.0  0.0  0.0     0.142002  0.0       0.0       
 1.0  0.848059   0.719203    0.0  1.0  0.0     0.0       0.719203  0.0       
 1.0  0.0967081  0.00935246  0.0  0.0  1.0     0.0       0.0       0.00935246
 1.0  0.92813    0.861425    0.0  0.0  0.0     0.0       0.0       0.0       

In [86]:
coefnames(f2_concrete.rhs)

12-element Array{String,1}:
 "(Intercept)" 
 "a"           
 "a ^ 2"       
 "b: R"        
 "b: S"        
 "b: T"        
 "a & b: R"    
 "a & b: S"    
 "a & b: T"    
 "a ^ 2 & b: R"
 "a ^ 2 & b: S"
 "a ^ 2 & b: T"

### Advanced: making the ordinary special

You may have noticed that `zercocorr` and `|` were not included in the list of
special syntax above.  StatsModels.jl provides a method to add special syntax
for the `@formula` that's specific to certain models.  This works using the
standard Julia techniques of multiple dispatch, by providing methods that
intercept `apply_schema` for particular combinations of functions, schema, and
context (model type), like so:

```
function StatsModels.apply_schema(
    t::FunctionTerm{typeof(|)},
    schema::StatsModels.FullRank,
    Mod::Type{<:MixedModel},
)
    schema = StatsModels.FullRank(schema.schema)
    lhs, rhs = t.args_parsed
    if !StatsModels.hasintercept(lhs) && !StatsModels.omitsintercept(lhs)
        lhs = InterceptTerm{true}() + lhs
    end
    lhs, rhs = apply_schema.((lhs, rhs), Ref(schema), Mod)
    RandomEffectsTerm(MatrixTerm(lhs), rhs)
end
```

There's a simpler [example in the StatsModels
docs](https://juliastats.org/StatsModels.jl/stable/internals/#An-example-of-custom-syntax:-poly-1)
which adds a `poly(x, n)` syntax for polynomial regression.